In [1]:
import tensorflow as tf
import numpy as np


class MNISTLoader():
    def __init__(self):
        mnist = tf.keras.datasets.mnist
        (self.train_data, self.train_label), (self.test_data,
                                              self.test_label) = mnist.load_data()
        # MNIST中的图像默认为uint8（0-255的数字）
        # 以下代码将其归一化到0-1之间的浮点数，并在最后增加一维作为颜色通道
        self.train_data = np.expand_dims(
            self.train_data.astype(
                np.float32) / 255.0,
            axis=-1)  											# [60000, 28, 28, 1]
        self.test_data = np.expand_dims(
            self.test_data.astype(
                np.float32) / 255.0,
            axis=-1)   											# [10000, 28, 28, 1]
        self.train_label = self.train_label.astype(np.int32)    # [60000]
        self.test_label = self.test_label.astype(np.int32)      # [10000]
        self.num_train_data, self.num_test_data = self.train_data.shape[
            0], self.test_data.shape[0]

    def get_batch(self, batch_size):
        # 从数据集中随机取出batch_size个元素并返回
        index = np.random.randint(0, np.shape(self.train_data)[0], batch_size)
        return self.train_data[index, :], self.train_label[index]


class CNN(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.conv1 = tf.keras.layers.Conv2D(
            filters=6,             # 卷积层神经元（卷积核）数目
            kernel_size=[5, 5],     # 感受野大小
            padding='valid',         # padding策略（vaild 或 same）
            strides=(1, 1),
            activation=tf.nn.relu   # 激活函数
        )
        self.pool1 = tf.keras.layers.MaxPool2D(pool_size=[2, 2], strides=2)
        self.conv2 = tf.keras.layers.Conv2D(
            filters=16,
            kernel_size=[5, 5],
            padding='valid',
            strides=(1, 1),
            activation=tf.nn.relu
        )
        self.pool2 = tf.keras.layers.MaxPool2D(pool_size=[2, 2], strides=2)
        self.flatten = tf.keras.layers.Flatten()
        # 等价于self.flatten = tf.keras.layers.Reshape(target_shape=(4 * 4 * 16,))
        
        self.dense1 = tf.keras.layers.Dense(units=120, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(units=84, activation=tf.nn.relu)
        self.dense3 = tf.keras.layers.Dense(units=10)

    def call(self, inputs):
        x = self.conv1(inputs)                  # [batch_size, 24, 24, 6]
        x = self.pool1(x)                       # [batch_size, 12, 12, 6]
        x = self.conv2(x)                       # [batch_size, 8, 8, 16]
        x = self.pool2(x)                       # [batch_size, 4, 4, 16]
        x = self.flatten(x)                     # [batch_size, 5 * 5 * 16]
        x = self.dense1(x)                      # [batch_size, 120]
        x = self.dense2(x)                      # [batch_size, 84]
        x = self.dense3(x)                      # [batch_size, 10]
        output = tf.nn.softmax(x)
        return output


num_epochs = 5			# 训练轮数
batch_size = 50			# 批大小
learning_rate = 0.001  # 学习率

tf.compat.v1.enable_eager_execution()
model = CNN()														# 实例化模型
data_loader = MNISTLoader()											# 数据载入
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)  # 实例化优化器


num_batches = int(data_loader.num_train_data // batch_size * num_epochs)
for batch_index in range(num_batches):
                # 随机取一批训练数据
    X, y = data_loader.get_batch(batch_size)
    with tf.GradientTape() as tape:
                # 计算模型预测值
        y_pred = model(X)
        # 计算损失函数
        loss = tf.keras.losses.sparse_categorical_crossentropy(
            y_true=y, y_pred=y_pred)
        loss = tf.reduce_mean(loss)
        print("batch %d: loss %f" % (batch_index, loss.numpy()))
    # 计算模型变量的导数
    grads = tape.gradient(loss, model.variables)
    # 优化器的使用
    optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables))


# 评估器
sparse_categorical_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
# 迭代轮数
num_batches = int(data_loader.num_test_data // batch_size)
for batch_index in range(num_batches):
    start_index, end_index = batch_index * \
        batch_size, (batch_index + 1) * batch_size
    # 模型预测的结果
    y_pred = model.predict(data_loader.test_data[start_index: end_index])
    sparse_categorical_accuracy.update_state(
        y_true=data_loader.test_label[start_index: end_index], y_pred=y_pred)
print("test accuracy: %f" % sparse_categorical_accuracy.result())

batch 0: loss 2.345943
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
batch 1: loss 2.285000
batch 2: loss 2.274218
batch 3: loss 2.303022
batch 4: loss 2.236306
batch 5: loss 2.236228
batch 6: loss 2.235421
batch 7: loss 2.260436
batch 8: loss 2.262228
batch 9: loss 2.200608
batch 10: loss 2.170460
batch 11: loss 2.200849
batch 12: loss 2.135831
batch 13: loss 2.110245
batch 14: loss 2.103041
batch 15: loss 2.019065
batch 16: loss 2.041911
batch 17: loss 2.041267
batch 18: loss 1.934611
batch 19: loss 2.013029
batch 20: loss 1.925704
batch 21: loss 1.814292
batch 22: loss 1.893494
batch 23: loss 1.840951
batch 24: loss 1.752017
batch 25: loss 1.772362
batch 26: loss 1.603958
batch 27: loss 1.528387
batch 28: loss 1.529063
batch 29: loss 1.305827
batch 30: loss 1.436641
batch 31: loss 1.232399
batch 32: loss 1.237342
batch 33: loss 1.157573
batch 34: loss 1.164656
batch 35: loss 1.037933
batch 36: loss 1.031432
batch 37: loss 1.194595
batc

batch 334: loss 0.221177
batch 335: loss 0.073506
batch 336: loss 0.218694
batch 337: loss 0.133488
batch 338: loss 0.271203
batch 339: loss 0.303894
batch 340: loss 0.241193
batch 341: loss 0.050807
batch 342: loss 0.042795
batch 343: loss 0.112537
batch 344: loss 0.275035
batch 345: loss 0.385304
batch 346: loss 0.239719
batch 347: loss 0.162554
batch 348: loss 0.179750
batch 349: loss 0.217635
batch 350: loss 0.128865
batch 351: loss 0.292537
batch 352: loss 0.166928
batch 353: loss 0.060053
batch 354: loss 0.098479
batch 355: loss 0.143616
batch 356: loss 0.129960
batch 357: loss 0.108448
batch 358: loss 0.126152
batch 359: loss 0.056729
batch 360: loss 0.267567
batch 361: loss 0.058357
batch 362: loss 0.194050
batch 363: loss 0.339890
batch 364: loss 0.197888
batch 365: loss 0.136456
batch 366: loss 0.261516
batch 367: loss 0.102022
batch 368: loss 0.067364
batch 369: loss 0.095463
batch 370: loss 0.050464
batch 371: loss 0.131278
batch 372: loss 0.315287
batch 373: loss 0.200107


batch 676: loss 0.151222
batch 677: loss 0.093073
batch 678: loss 0.048440
batch 679: loss 0.077059
batch 680: loss 0.137761
batch 681: loss 0.043368
batch 682: loss 0.324410
batch 683: loss 0.094392
batch 684: loss 0.080200
batch 685: loss 0.106822
batch 686: loss 0.139680
batch 687: loss 0.119289
batch 688: loss 0.082390
batch 689: loss 0.072970
batch 690: loss 0.186292
batch 691: loss 0.084754
batch 692: loss 0.201283
batch 693: loss 0.082610
batch 694: loss 0.059090
batch 695: loss 0.140267
batch 696: loss 0.161826
batch 697: loss 0.203288
batch 698: loss 0.157997
batch 699: loss 0.068928
batch 700: loss 0.078601
batch 701: loss 0.127698
batch 702: loss 0.038255
batch 703: loss 0.198754
batch 704: loss 0.356167
batch 705: loss 0.104753
batch 706: loss 0.105089
batch 707: loss 0.129646
batch 708: loss 0.115477
batch 709: loss 0.161619
batch 710: loss 0.042329
batch 711: loss 0.067798
batch 712: loss 0.076957
batch 713: loss 0.181579
batch 714: loss 0.287608
batch 715: loss 0.210316


batch 1015: loss 0.039844
batch 1016: loss 0.097139
batch 1017: loss 0.088529
batch 1018: loss 0.064467
batch 1019: loss 0.025586
batch 1020: loss 0.250606
batch 1021: loss 0.077298
batch 1022: loss 0.081427
batch 1023: loss 0.035024
batch 1024: loss 0.065439
batch 1025: loss 0.042797
batch 1026: loss 0.076166
batch 1027: loss 0.060410
batch 1028: loss 0.066730
batch 1029: loss 0.172116
batch 1030: loss 0.026110
batch 1031: loss 0.030396
batch 1032: loss 0.025383
batch 1033: loss 0.045429
batch 1034: loss 0.042770
batch 1035: loss 0.005953
batch 1036: loss 0.191833
batch 1037: loss 0.078826
batch 1038: loss 0.021886
batch 1039: loss 0.201954
batch 1040: loss 0.214736
batch 1041: loss 0.036115
batch 1042: loss 0.146955
batch 1043: loss 0.082723
batch 1044: loss 0.069079
batch 1045: loss 0.080465
batch 1046: loss 0.030414
batch 1047: loss 0.117386
batch 1048: loss 0.121728
batch 1049: loss 0.009739
batch 1050: loss 0.034780
batch 1051: loss 0.018037
batch 1052: loss 0.063590
batch 1053: 

batch 1336: loss 0.133617
batch 1337: loss 0.188737
batch 1338: loss 0.028977
batch 1339: loss 0.124835
batch 1340: loss 0.020479
batch 1341: loss 0.035253
batch 1342: loss 0.106775
batch 1343: loss 0.116641
batch 1344: loss 0.010591
batch 1345: loss 0.065115
batch 1346: loss 0.073953
batch 1347: loss 0.142324
batch 1348: loss 0.023129
batch 1349: loss 0.073422
batch 1350: loss 0.153610
batch 1351: loss 0.186862
batch 1352: loss 0.094442
batch 1353: loss 0.272265
batch 1354: loss 0.074473
batch 1355: loss 0.018328
batch 1356: loss 0.126647
batch 1357: loss 0.016783
batch 1358: loss 0.156249
batch 1359: loss 0.058744
batch 1360: loss 0.041630
batch 1361: loss 0.009314
batch 1362: loss 0.031801
batch 1363: loss 0.021446
batch 1364: loss 0.122874
batch 1365: loss 0.054868
batch 1366: loss 0.046753
batch 1367: loss 0.070108
batch 1368: loss 0.039603
batch 1369: loss 0.152243
batch 1370: loss 0.137719
batch 1371: loss 0.045136
batch 1372: loss 0.054388
batch 1373: loss 0.263402
batch 1374: 

...

batch 4550: loss 0.046167
batch 4551: loss 0.001628
batch 4552: loss 0.003621
batch 4553: loss 0.003065
batch 4554: loss 0.025091
batch 4555: loss 0.089891
batch 4556: loss 0.002133
batch 4557: loss 0.014335
batch 4558: loss 0.016209
batch 4559: loss 0.003787
batch 4560: loss 0.011633
batch 4561: loss 0.015054
batch 4562: loss 0.043703
batch 4563: loss 0.003580
batch 4564: loss 0.037331
batch 4565: loss 0.131761
batch 4566: loss 0.013488
batch 4567: loss 0.019688
batch 4568: loss 0.075079
batch 4569: loss 0.094576
batch 4570: loss 0.075327
batch 4571: loss 0.161080
batch 4572: loss 0.037178
batch 4573: loss 0.101201
batch 4574: loss 0.054389
batch 4575: loss 0.002609
batch 4576: loss 0.040282
batch 4577: loss 0.018005
batch 4578: loss 0.033822
batch 4579: loss 0.049231
batch 4580: loss 0.055863
batch 4581: loss 0.022052
batch 4582: loss 0.025785
batch 4583: loss 0.015579
batch 4584: loss 0.032782
batch 4585: loss 0.004552
batch 4586: loss 0.214940
batch 4587: loss 0.012466
batch 4588: 

batch 4873: loss 0.091193
batch 4874: loss 0.003831
batch 4875: loss 0.054410
batch 4876: loss 0.040108
batch 4877: loss 0.008164
batch 4878: loss 0.012349
batch 4879: loss 0.051952
batch 4880: loss 0.022178
batch 4881: loss 0.150353
batch 4882: loss 0.003349
batch 4883: loss 0.029155
batch 4884: loss 0.047362
batch 4885: loss 0.001966
batch 4886: loss 0.023643
batch 4887: loss 0.114947
batch 4888: loss 0.045162
batch 4889: loss 0.043826
batch 4890: loss 0.002552
batch 4891: loss 0.008164
batch 4892: loss 0.002181
batch 4893: loss 0.118620
batch 4894: loss 0.068986
batch 4895: loss 0.014622
batch 4896: loss 0.024123
batch 4897: loss 0.048031
batch 4898: loss 0.003541
batch 4899: loss 0.030804
batch 4900: loss 0.004035
batch 4901: loss 0.001595
batch 4902: loss 0.009391
batch 4903: loss 0.136496
batch 4904: loss 0.010123
batch 4905: loss 0.000602
batch 4906: loss 0.025985
batch 4907: loss 0.019123
batch 4908: loss 0.054617
batch 4909: loss 0.003877
batch 4910: loss 0.006600
batch 4911: 

batch 5190: loss 0.029350
batch 5191: loss 0.004017
batch 5192: loss 0.024813
batch 5193: loss 0.149536
batch 5194: loss 0.068246
batch 5195: loss 0.017029
batch 5196: loss 0.016002
batch 5197: loss 0.024048
batch 5198: loss 0.038608
batch 5199: loss 0.001955
batch 5200: loss 0.012252
batch 5201: loss 0.008673
batch 5202: loss 0.023007
batch 5203: loss 0.086548
batch 5204: loss 0.020153
batch 5205: loss 0.002444
batch 5206: loss 0.130755
batch 5207: loss 0.007567
batch 5208: loss 0.087620
batch 5209: loss 0.025055
batch 5210: loss 0.001633
batch 5211: loss 0.033415
batch 5212: loss 0.056753
batch 5213: loss 0.004353
batch 5214: loss 0.104524
batch 5215: loss 0.002619
batch 5216: loss 0.012182
batch 5217: loss 0.033419
batch 5218: loss 0.008165
batch 5219: loss 0.163272
batch 5220: loss 0.001410
batch 5221: loss 0.095169
batch 5222: loss 0.016860
batch 5223: loss 0.047503
batch 5224: loss 0.014046
batch 5225: loss 0.119407
batch 5226: loss 0.060589
batch 5227: loss 0.003084
batch 5228: 

batch 5507: loss 0.016129
batch 5508: loss 0.002418
batch 5509: loss 0.005956
batch 5510: loss 0.003238
batch 5511: loss 0.071688
batch 5512: loss 0.005352
batch 5513: loss 0.061428
batch 5514: loss 0.030482
batch 5515: loss 0.000464
batch 5516: loss 0.110201
batch 5517: loss 0.006610
batch 5518: loss 0.000444
batch 5519: loss 0.006709
batch 5520: loss 0.014071
batch 5521: loss 0.002795
batch 5522: loss 0.017820
batch 5523: loss 0.012595
batch 5524: loss 0.026588
batch 5525: loss 0.004459
batch 5526: loss 0.050555
batch 5527: loss 0.104418
batch 5528: loss 0.017854
batch 5529: loss 0.014657
batch 5530: loss 0.000947
batch 5531: loss 0.042823
batch 5532: loss 0.094834
batch 5533: loss 0.207785
batch 5534: loss 0.031945
batch 5535: loss 0.006291
batch 5536: loss 0.003522
batch 5537: loss 0.020556
batch 5538: loss 0.034824
batch 5539: loss 0.023408
batch 5540: loss 0.001714
batch 5541: loss 0.017331
batch 5542: loss 0.003224
batch 5543: loss 0.004078
batch 5544: loss 0.001953
batch 5545: 

batch 5830: loss 0.001728
batch 5831: loss 0.001751
batch 5832: loss 0.001669
batch 5833: loss 0.006899
batch 5834: loss 0.052040
batch 5835: loss 0.096797
batch 5836: loss 0.018806
batch 5837: loss 0.022938
batch 5838: loss 0.077750
batch 5839: loss 0.075518
batch 5840: loss 0.004628
batch 5841: loss 0.024479
batch 5842: loss 0.012665
batch 5843: loss 0.029083
batch 5844: loss 0.058220
batch 5845: loss 0.018322
batch 5846: loss 0.028815
batch 5847: loss 0.002842
batch 5848: loss 0.188149
batch 5849: loss 0.001908
batch 5850: loss 0.006224
batch 5851: loss 0.001828
batch 5852: loss 0.016758
batch 5853: loss 0.039270
batch 5854: loss 0.008839
batch 5855: loss 0.002602
batch 5856: loss 0.028990
batch 5857: loss 0.006500
batch 5858: loss 0.001775
batch 5859: loss 0.024068
batch 5860: loss 0.004627
batch 5861: loss 0.010981
batch 5862: loss 0.330913
batch 5863: loss 0.036163
batch 5864: loss 0.043365
batch 5865: loss 0.072017
batch 5866: loss 0.156387
batch 5867: loss 0.028961
batch 5868: 

In [7]:
import tensorflow as tf
import numpy as np


class MNISTLoader():
    def __init__(self):
        mnist = tf.keras.datasets.mnist
        (self.train_data, self.train_label), (self.test_data,
                                              self.test_label) = mnist.load_data()
        # MNIST中的图像默认为uint8（0-255的数字）
        # 以下代码将其归一化到0-1之间的浮点数，并在最后增加一维作为颜色通道
        self.train_data = np.expand_dims(
            self.train_data.astype(
                np.float32) / 255.0,
            axis=-1)  											# [60000, 28, 28, 1]
        self.test_data = np.expand_dims(
            self.test_data.astype(
                np.float32) / 255.0,
            axis=-1)   											# [10000, 28, 28, 1]
        self.train_label = self.train_label.astype(np.int32)    # [60000]
        self.test_label = self.test_label.astype(np.int32)      # [10000]
        self.num_train_data, self.num_test_data = self.train_data.shape[
            0], self.test_data.shape[0]

    def get_batch(self, batch_size):
        # 从数据集中随机取出batch_size个元素并返回
        index = np.random.randint(0, np.shape(self.train_data)[0], batch_size)
        return self.train_data[index, :], self.train_label[index]


class MLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        # Flatten层将除第一维（batch_size）以外的维度展平
        self.flatten = tf.keras.layers.Flatten()
        # 全连接层
        self.dense1 = tf.keras.layers.Dense(units=100, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(units=10)

    def call(self, inputs):         # [batch_size, 28, 28, 1]
        x = self.flatten(inputs)    # [batch_size, 784]
        x = self.dense1(x)          # [batch_size, 100]
        x = self.dense2(x)          # [batch_size, 10]
        output = tf.nn.softmax(x)
        return output


num_epochs = 5			# 训练轮数
batch_size = 50			# 批大小
learning_rate = 0.001  # 学习率

tf.compat.v1.enable_eager_execution()
model = MLP()														# 实例化模型
data_loader = MNISTLoader()											# 数据载入
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)  # 实例化优化器


num_batches = int(data_loader.num_train_data // batch_size * num_epochs)
for batch_index in range(num_batches):
                # 随机取一批训练数据
    X, y = data_loader.get_batch(batch_size)
    with tf.GradientTape() as tape:
                # 计算模型预测值
        y_pred = model(X)
        # 计算损失函数
        loss = tf.keras.losses.sparse_categorical_crossentropy(
            y_true=y, y_pred=y_pred)
        loss = tf.reduce_mean(loss)
        print("batch %d: loss %f" % (batch_index, loss.numpy()))
    # 计算模型变量的导数
    grads = tape.gradient(loss, model.variables)
    # 优化器的使用
    optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables))


# 评估器
sparse_categorical_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
# 迭代轮数
num_batches = int(data_loader.num_test_data // batch_size)
for batch_index in range(num_batches):
    start_index, end_index = batch_index * \
        batch_size, (batch_index + 1) * batch_size
    # 模型预测的结果
    y_pred = model.predict(data_loader.test_data[start_index: end_index])
    sparse_categorical_accuracy.update_state(
        y_true=data_loader.test_label[start_index: end_index], y_pred=y_pred)
print("test accuracy: %f" % sparse_categorical_accuracy.result())

batch 0: loss 2.407920
batch 1: loss 2.322922
batch 2: loss 2.178479
batch 3: loss 2.122109
batch 4: loss 2.030552
batch 5: loss 2.047362
batch 6: loss 2.051404
batch 7: loss 1.739664
batch 8: loss 1.922837
batch 9: loss 1.694568
batch 10: loss 1.737622
batch 11: loss 1.823888
batch 12: loss 1.515856
batch 13: loss 1.497776
batch 14: loss 1.521414
batch 15: loss 1.407023
batch 16: loss 1.405943
batch 17: loss 1.438476
batch 18: loss 1.525484
batch 19: loss 1.190447
batch 20: loss 1.178490
batch 21: loss 1.238236
batch 22: loss 1.174747
batch 23: loss 0.924776
batch 24: loss 1.076185
batch 25: loss 0.881536
batch 26: loss 1.164899
batch 27: loss 0.865520
batch 28: loss 0.929586
batch 29: loss 0.992541
batch 30: loss 0.848217
batch 31: loss 1.069227
batch 32: loss 0.927899
batch 33: loss 0.891414
batch 34: loss 0.888311
batch 35: loss 0.835835
batch 36: loss 0.786759
batch 37: loss 0.828357
batch 38: loss 0.624680
batch 39: loss 0.722084
batch 40: loss 0.876669
batch 41: loss 0.751052
ba

batch 367: loss 0.484279
batch 368: loss 0.155618
batch 369: loss 0.206373
batch 370: loss 0.283435
batch 371: loss 0.133975
batch 372: loss 0.278696
batch 373: loss 0.128512
batch 374: loss 0.245129
batch 375: loss 0.234795
batch 376: loss 0.288536
batch 377: loss 0.219372
batch 378: loss 0.146167
batch 379: loss 0.249781
batch 380: loss 0.102842
batch 381: loss 0.406573
batch 382: loss 0.182159
batch 383: loss 0.308016
batch 384: loss 0.331656
batch 385: loss 0.215935
batch 386: loss 0.333671
batch 387: loss 0.194762
batch 388: loss 0.179569
batch 389: loss 0.371933
batch 390: loss 0.211955
batch 391: loss 0.282879
batch 392: loss 0.354025
batch 393: loss 0.379476
batch 394: loss 0.545513
batch 395: loss 0.328715
batch 396: loss 0.244112
batch 397: loss 0.201320
batch 398: loss 0.180344
batch 399: loss 0.381733
batch 400: loss 0.151939
batch 401: loss 0.223471
batch 402: loss 0.484152
batch 403: loss 0.337552
batch 404: loss 0.179280
batch 405: loss 0.126171
batch 406: loss 0.243966


batch 738: loss 0.237308
batch 739: loss 0.107558
batch 740: loss 0.223260
batch 741: loss 0.195071
batch 742: loss 0.153261
batch 743: loss 0.157465
batch 744: loss 0.392301
batch 745: loss 0.113368
batch 746: loss 0.261145
batch 747: loss 0.120354
batch 748: loss 0.400603
batch 749: loss 0.110944
batch 750: loss 0.218047
batch 751: loss 0.081653
batch 752: loss 0.155669
batch 753: loss 0.121146
batch 754: loss 0.205973
batch 755: loss 0.165607
batch 756: loss 0.176001
batch 757: loss 0.156808
batch 758: loss 0.280559
batch 759: loss 0.323874
batch 760: loss 0.138478
batch 761: loss 0.070463
batch 762: loss 0.235215
batch 763: loss 0.257653
batch 764: loss 0.326926
batch 765: loss 0.144216
batch 766: loss 0.203325
batch 767: loss 0.144116
batch 768: loss 0.106808
batch 769: loss 0.167671
batch 770: loss 0.175408
batch 771: loss 0.204005
batch 772: loss 0.172100
batch 773: loss 0.256376
batch 774: loss 0.168280
batch 775: loss 0.224069
batch 776: loss 0.268161
batch 777: loss 0.303048


batch 1109: loss 0.120972
batch 1110: loss 0.047079
batch 1111: loss 0.264990
batch 1112: loss 0.087782
batch 1113: loss 0.126169
batch 1114: loss 0.071699
batch 1115: loss 0.201793
batch 1116: loss 0.087484
batch 1117: loss 0.189652
batch 1118: loss 0.120147
batch 1119: loss 0.060832
batch 1120: loss 0.113637
batch 1121: loss 0.169553
batch 1122: loss 0.083143
batch 1123: loss 0.164878
batch 1124: loss 0.088793
batch 1125: loss 0.362188
batch 1126: loss 0.079830
batch 1127: loss 0.186088
batch 1128: loss 0.183557
batch 1129: loss 0.032775
batch 1130: loss 0.207188
batch 1131: loss 0.239254
batch 1132: loss 0.254309
batch 1133: loss 0.163587
batch 1134: loss 0.170123
batch 1135: loss 0.208264
batch 1136: loss 0.275319
batch 1137: loss 0.172443
batch 1138: loss 0.147331
batch 1139: loss 0.076212
batch 1140: loss 0.091499
batch 1141: loss 0.056409
batch 1142: loss 0.281907
batch 1143: loss 0.187194
batch 1144: loss 0.233692
batch 1145: loss 0.133608
batch 1146: loss 0.227772
batch 1147: 

batch 1429: loss 0.070968
batch 1430: loss 0.051451
batch 1431: loss 0.092623
batch 1432: loss 0.092405
batch 1433: loss 0.168169
batch 1434: loss 0.055945
batch 1435: loss 0.194143
batch 1436: loss 0.092508
batch 1437: loss 0.215370
batch 1438: loss 0.241155
batch 1439: loss 0.101344
batch 1440: loss 0.039551
batch 1441: loss 0.090364
batch 1442: loss 0.108937
batch 1443: loss 0.237991
batch 1444: loss 0.122503
batch 1445: loss 0.157715
batch 1446: loss 0.290161
batch 1447: loss 0.110641
batch 1448: loss 0.201851
batch 1449: loss 0.117319
batch 1450: loss 0.121817
batch 1451: loss 0.191538
batch 1452: loss 0.120788
batch 1453: loss 0.299664
batch 1454: loss 0.242682
batch 1455: loss 0.074619
batch 1456: loss 0.050715
batch 1457: loss 0.093201
batch 1458: loss 0.203494
batch 1459: loss 0.127302
batch 1460: loss 0.112130
batch 1461: loss 0.227508
batch 1462: loss 0.061247
batch 1463: loss 0.047457
batch 1464: loss 0.107452
batch 1465: loss 0.079351
batch 1466: loss 0.265846
batch 1467: 

...

batch 4611: loss 0.035502
batch 4612: loss 0.047798
batch 4613: loss 0.075666
batch 4614: loss 0.072362
batch 4615: loss 0.128268
batch 4616: loss 0.065772
batch 4617: loss 0.030188
batch 4618: loss 0.274844
batch 4619: loss 0.047927
batch 4620: loss 0.013447
batch 4621: loss 0.232143
batch 4622: loss 0.016920
batch 4623: loss 0.156053
batch 4624: loss 0.057337
batch 4625: loss 0.143675
batch 4626: loss 0.102502
batch 4627: loss 0.121378
batch 4628: loss 0.145934
batch 4629: loss 0.037328
batch 4630: loss 0.156869
batch 4631: loss 0.108092
batch 4632: loss 0.128253
batch 4633: loss 0.034705
batch 4634: loss 0.016039
batch 4635: loss 0.045712
batch 4636: loss 0.055124
batch 4637: loss 0.030349
batch 4638: loss 0.017877
batch 4639: loss 0.025327
batch 4640: loss 0.037914
batch 4641: loss 0.036096
batch 4642: loss 0.025176
batch 4643: loss 0.052232
batch 4644: loss 0.183299
batch 4645: loss 0.320567
batch 4646: loss 0.097335
batch 4647: loss 0.054490
batch 4648: loss 0.010697
batch 4649: 

batch 4929: loss 0.066072
batch 4930: loss 0.058892
batch 4931: loss 0.089601
batch 4932: loss 0.048842
batch 4933: loss 0.015535
batch 4934: loss 0.021890
batch 4935: loss 0.263081
batch 4936: loss 0.066258
batch 4937: loss 0.016711
batch 4938: loss 0.065260
batch 4939: loss 0.022860
batch 4940: loss 0.115595
batch 4941: loss 0.043811
batch 4942: loss 0.025627
batch 4943: loss 0.127409
batch 4944: loss 0.083734
batch 4945: loss 0.047726
batch 4946: loss 0.024794
batch 4947: loss 0.228538
batch 4948: loss 0.069419
batch 4949: loss 0.042528
batch 4950: loss 0.078789
batch 4951: loss 0.044072
batch 4952: loss 0.049974
batch 4953: loss 0.031382
batch 4954: loss 0.135151
batch 4955: loss 0.045451
batch 4956: loss 0.046803
batch 4957: loss 0.088160
batch 4958: loss 0.074322
batch 4959: loss 0.024459
batch 4960: loss 0.026029
batch 4961: loss 0.151435
batch 4962: loss 0.043433
batch 4963: loss 0.053654
batch 4964: loss 0.221403
batch 4965: loss 0.028275
batch 4966: loss 0.019378
batch 4967: 

batch 5247: loss 0.042014
batch 5248: loss 0.108673
batch 5249: loss 0.055316
batch 5250: loss 0.007387
batch 5251: loss 0.121413
batch 5252: loss 0.068099
batch 5253: loss 0.166907
batch 5254: loss 0.039210
batch 5255: loss 0.039095
batch 5256: loss 0.113044
batch 5257: loss 0.024571
batch 5258: loss 0.027464
batch 5259: loss 0.012411
batch 5260: loss 0.007926
batch 5261: loss 0.130993
batch 5262: loss 0.011983
batch 5263: loss 0.024073
batch 5264: loss 0.025376
batch 5265: loss 0.110410
batch 5266: loss 0.053886
batch 5267: loss 0.009495
batch 5268: loss 0.083679
batch 5269: loss 0.156639
batch 5270: loss 0.214116
batch 5271: loss 0.109712
batch 5272: loss 0.029155
batch 5273: loss 0.143052
batch 5274: loss 0.054099
batch 5275: loss 0.105971
batch 5276: loss 0.022842
batch 5277: loss 0.007428
batch 5278: loss 0.024253
batch 5279: loss 0.020896
batch 5280: loss 0.132094
batch 5281: loss 0.071402
batch 5282: loss 0.050719
batch 5283: loss 0.143638
batch 5284: loss 0.065901
batch 5285: 

batch 5565: loss 0.011105
batch 5566: loss 0.037849
batch 5567: loss 0.153045
batch 5568: loss 0.025423
batch 5569: loss 0.015855
batch 5570: loss 0.170732
batch 5571: loss 0.044757
batch 5572: loss 0.017436
batch 5573: loss 0.077662
batch 5574: loss 0.036282
batch 5575: loss 0.016085
batch 5576: loss 0.054106
batch 5577: loss 0.035839
batch 5578: loss 0.035625
batch 5579: loss 0.030687
batch 5580: loss 0.014374
batch 5581: loss 0.007096
batch 5582: loss 0.045953
batch 5583: loss 0.020497
batch 5584: loss 0.095947
batch 5585: loss 0.097406
batch 5586: loss 0.033734
batch 5587: loss 0.013951
batch 5588: loss 0.007298
batch 5589: loss 0.012896
batch 5590: loss 0.021807
batch 5591: loss 0.023398
batch 5592: loss 0.078325
batch 5593: loss 0.023942
batch 5594: loss 0.068537
batch 5595: loss 0.023944
batch 5596: loss 0.025445
batch 5597: loss 0.149422
batch 5598: loss 0.087271
batch 5599: loss 0.040080
batch 5600: loss 0.042048
batch 5601: loss 0.036838
batch 5602: loss 0.039286
batch 5603: 

batch 5883: loss 0.040999
batch 5884: loss 0.029382
batch 5885: loss 0.136239
batch 5886: loss 0.019008
batch 5887: loss 0.008161
batch 5888: loss 0.149772
batch 5889: loss 0.126956
batch 5890: loss 0.041262
batch 5891: loss 0.046572
batch 5892: loss 0.061230
batch 5893: loss 0.017774
batch 5894: loss 0.105821
batch 5895: loss 0.036047
batch 5896: loss 0.113940
batch 5897: loss 0.084893
batch 5898: loss 0.009957
batch 5899: loss 0.012663
batch 5900: loss 0.039548
batch 5901: loss 0.179154
batch 5902: loss 0.022979
batch 5903: loss 0.117766
batch 5904: loss 0.021497
batch 5905: loss 0.121685
batch 5906: loss 0.050522
batch 5907: loss 0.006712
batch 5908: loss 0.179060
batch 5909: loss 0.071604
batch 5910: loss 0.114258
batch 5911: loss 0.046544
batch 5912: loss 0.068045
batch 5913: loss 0.116569
batch 5914: loss 0.039956
batch 5915: loss 0.118000
batch 5916: loss 0.067495
batch 5917: loss 0.058126
batch 5918: loss 0.103099
batch 5919: loss 0.024157
batch 5920: loss 0.058935
batch 5921: 